In [7]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>

<article class="day-desc read-aloud"><h2>--- Day 23: Amphipod ---</h2><p>A group of <a href="https://en.wikipedia.org/wiki/Amphipoda" target="_blank">amphipods</a> notice your fancy submarine and flag you down. "With such an impressive shell," one amphipod <span title="What? You didn't know amphipods can talk?">says</span>, "surely you can help us with a question that has stumped our best scientists."</p>
<p>They go on to explain that a group of timid, stubborn amphipods live in a nearby burrow. Four types of amphipods live there: <em>Amber</em> (<code>A</code>), <em>Bronze</em> (<code>B</code>), <em>Copper</em> (<code>C</code>), and <em>Desert</em> (<code>D</code>). They live in a burrow that consists of a <em>hallway</em> and four <em>side rooms</em>. The side rooms are initially full of amphipods, and the hallway is initially empty.</p>
<p>They give you a <em>diagram of the situation</em> (your puzzle input), including locations of each amphipod (<code>A</code>, <code>B</code>, <code>C</code>, or <code>D</code>, each of which is occupying an otherwise open space), walls (<code>#</code>), and open space (<code>.</code>).</p>
<p>For example:</p>
<pre><code>#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
</code></pre>
<p>The amphipods would like a method to organize every amphipod into side rooms so that each side room contains one type of amphipod and the types are sorted <code>A</code>-<code>D</code> going left to right, like this:</p>
<pre><code>#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
</code></pre>
<p>Amphipods can move up, down, left, or right so long as they are moving into an unoccupied open space. Each type of amphipod requires a different amount of <em>energy</em> to move one step: Amber amphipods require <code>1</code> energy per step, Bronze amphipods require <code>10</code> energy, Copper amphipods require <code>100</code>, and Desert ones require <code>1000</code>. The amphipods would like you to find a way to organize the amphipods that requires the <em>least total energy</em>.</p>
<p>However, because they are timid and stubborn, the amphipods have some extra rules:</p>
<ul>
<li>Amphipods will never <em>stop on the space immediately outside any room</em>. They can move into that space so long as they immediately continue moving. (Specifically, this refers to the four open spaces in the hallway that are directly above an amphipod starting position.)</li>
<li>Amphipods will never <em>move from the hallway into a room</em> unless that room is their destination room <em>and</em> that room contains no amphipods which do not also have that room as their own destination. If an amphipod's starting room is not its destination room, it can stay in that room until it leaves the room. (For example, an Amber amphipod will not move from the hallway into the right three rooms, and will only move into the leftmost room if that room is empty or if it only contains other Amber amphipods.)</li>
<li>Once an amphipod stops moving in the hallway, <em>it will stay in that spot until it can move into a room</em>. (That is, once any amphipod starts moving, any other amphipods currently in the hallway are locked in place and will not move again until they can move fully into a room.)</li>
</ul>
<p>In the above example, the amphipods can be organized using a minimum of <code><em>12521</em></code> energy. One way to do this is shown below.</p>
<p>Starting configuration:</p>
<pre><code>#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
</code></pre>
<p>One Bronze amphipod moves into the hallway, taking 4 steps and using <code>40</code> energy:</p>
<pre><code>#############
#...B.......#
###B#C#.#D###
  #A#D#C#A#
  #########
</code></pre>
<p>The only Copper amphipod not in its side room moves there, taking 4 steps and using <code>400</code> energy:</p>
<pre><code>#############
#...B.......#
###B#.#C#D###
  #A#D#C#A#
  #########
</code></pre>
<p>A Desert amphipod moves out of the way, taking 3 steps and using <code>3000</code> energy, and then the Bronze amphipod takes its place, taking 3 steps and using <code>30</code> energy:</p>
<pre><code>#############
#.....D.....#
###B#.#C#D###
  #A#B#C#A#
  #########
</code></pre>
<p>The leftmost Bronze amphipod moves to its room using <code>40</code> energy:</p>
<pre><code>#############
#.....D.....#
###.#B#C#D###
  #A#B#C#A#
  #########
</code></pre>
<p>Both amphipods in the rightmost room move into the hallway, using <code>2003</code> energy in total:</p>
<pre><code>#############
#.....D.D.A.#
###.#B#C#.###
  #A#B#C#.#
  #########
</code></pre>
<p>Both Desert amphipods move into the rightmost room using <code>7000</code> energy:</p>
<pre><code>#############
#.........A.#
###.#B#C#D###
  #A#B#C#D#
  #########
</code></pre>
<p>Finally, the last Amber amphipod moves into its room, using <code>8</code> energy:</p>
<pre><code>#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
</code></pre>
<p><em>What is the least energy required to organize the amphipods?</em></p>
</article>


In [ ]:
from collections.abc import Iterator
from copy import deepcopy
from functools import total_ordering
from heapq import heappop, heappush
from more_itertools import minmax
from tabulate import tabulate


example = """
#############
#...........#
###B#C#B#D###
  #A#D#C#A#      
  #########    
"""


# #############
# #ab.c.d.e.fg#
# ###A#B#C#D### 0
#   #A#B#C#D#   1
#   #########
#


@total_ordering
class State:
    Nodes = {
        "A0": (2, 3),
        "A1": (3, 3),
        "B0": (2, 5),
        "B1": (3, 5),
        "C0": (2, 7),
        "C1": (3, 7),
        "D0": (2, 9),
        "D1": (3, 9),
        "a": (1, 1),
        "b": (1, 2),
        "c": (1, 4),
        "d": (1, 6),
        "e": (1, 8),
        "f": (1, 10),
        "g": (1, 11),
    }

    PointsNodes = {v: k for k, v in Nodes.items()}

    Cost = {"A": 1, "B": 10, "C": 100, "D": 1_000}

    def __init__(self, diagram: list[list[str]]):
        self.costs = 0
        self.rooms = {
            "A0": diagram[2][3],
            "A1": diagram[3][3],
            "B0": diagram[2][5],
            "B1": diagram[3][5],
            "C0": diagram[2][7],
            "C1": diagram[3][7],
            "D0": diagram[2][9],
            "D1": diagram[3][9],
        }

        self.hallway = {h: None for h in "abcdefg"}
        self.not_placed = {a: set() for a in "ABCD"}

        for room, a in self.rooms.items():
            if a == room[0] and room[1] == "1":
                continue
            if a == room[0] and room[1] == "0" and self.rooms[room[0] + "1"] == room[0]:
                continue
            self.not_placed[a].add(room)

        self.not_placed = dict((k, v) for k, v in self.not_placed.items() if v)

        self.diagram = diagram

    def is_goal(self) -> bool:
        return all(self.rooms[a + "1"] == self.rooms[a + "0"] == a for a in "ABCD")

    def move(self, verbose=False) -> Iterator[State]:
        for amph, nodes in self.not_placed.items():
            for node in nodes:
                if len(node) == 2:
                    yield from self.from_rooms(amph, node, verbose)
                else:
                    yield from self.from_hallway_to_rooms(amph, node, verbose)

    def from_rooms(self, amph: str, node: str, verbose=False) -> Iterator[State]:
        if node[1] == "1" and self.rooms[node[0] + "0"]:
            return
        if node[1] == "0" and self.rooms[node] == self.rooms[node[0] + "1"] == amph:
            return

        if self.rooms[amph + "1"] is None and self.rooms[amph + "0"] is None:
            room_to = amph + "1"
        else:
            room_to = amph + "0"

        if not self.rooms[room_to]:

            fr, to = minmax(self.Nodes[node][1], self.Nodes[room_to][1])
            if to < fr:
                fr, to = to + 1, fr + 1

            if all(not self.occupied(1, c) for c in range(fr, to)):
                if verbose:
                    print(f"From State R->R {node, room_to},")
                    print(self)
                state1 = deepcopy(self)
                state1.rooms[node] = None
                state1.rooms[room_to] = amph
                state1.not_placed[amph].remove(node)
                if not state1.not_placed[amph]:
                    del state1.not_placed[amph]
                state1.costs += self.cost(amph, node, room_to)
                if verbose:
                    print(f"To State R->R  {node, room_to}:")
                    print(state1)

                yield state1
                return

        for h, occupant in self.hallway.items():
            if occupant is None:
                fr, to = self.Nodes[node][1], self.Nodes[h][1]
                if fr > to:
                    fr, to = to + 1, fr + 1

                if any(self.occupied(1, c) for c in range(fr, to)):
                    continue

                if verbose:
                    print(f"From State R->H, {node, h}: ")
                    print(self)
                state1 = deepcopy(self)
                state1.rooms[node] = None
                state1.hallway[h] = amph
                state1.not_placed[amph].remove(node)
                state1.not_placed[amph].add(h)
                state1.costs += self.cost(amph, node, h)
                if verbose:
                    print(f"To State R ->H ,{ node, h}:")
                    print(state1)

                yield state1

    def from_hallway_to_rooms(
        self, amph: str, node: str, verbose=False
    ) -> Iterator[State]:
        fr, to = self.Nodes[node][1], self.Nodes[amph + "0"][1]
        if fr < to:
            fr, to = to + 1, fr + 1

            to += 1
        if any(self.occupied(1, c) for c in range(fr, to)):
            return

        if self.rooms[amph + "0"] is None and self.rooms[amph + "1"] is None:
            if verbose:
                print(f"From State H->R {node, amph + "1"}:")
                print(self)
            state1 = deepcopy(self)
            state1.hallway[node] = None
            state1.rooms[amph + "1"] = amph
            state1.not_placed[amph].remove(node)

            if not state1.not_placed[amph]:
                del state1.not_placed[amph]

            state1.costs += self.cost(amph, node, amph + "1")

            if verbose:
                print(f"To State H->R {node, amph + "1"}:")
                print(state1)

            yield state1
        elif self.rooms[amph + "0"] is None and self.rooms[amph + "1"] == amph:
            if verbose:
                print(f"From State H->R {node, amph + "0"}:")
                print(self)
            state1 = deepcopy(self)
            state1.hallway[node] = None
            state1.rooms[amph + "0"] = amph
            state1.not_placed[amph].remove(node)
            if not state1.not_placed[amph]:
                del state1.not_placed[amph]
            state1.costs += self.cost(amph, node, amph + "0")
            if verbose:
                print(f"To State H-< R {node, amph + "0"}:")
                print(state1)

            yield state1

    def occupied(self, r, c) -> bool:
        if (r, c) not in self.PointsNodes:
            return False
        node = self.PointsNodes[(r, c)]
        if len(node) == 2:
            return self.rooms[node] is not None
        return self.hallway[node] is not None

    def cost(self, amph: str, fr: str, to: str) -> int:
        return self.Cost[amph] * (
            abs(self.Nodes[fr][1] - self.Nodes[to][1])
            + ((self.Nodes[fr][0] - 1) if self.Nodes[fr][0] > 1 else 0)
            + ((self.Nodes[to][0] - 1) if self.Nodes[to][0] > 1 else 0)
        )

    def _get_str(self, r, c) -> str:
        if self.diagram[r][c] == "#":
            return "#"

        if (r, c) not in self.PointsNodes:
            return "."

        node = self.PointsNodes[(r, c)]
        if len(node) == 2:
            return self.rooms[node] if self.rooms[node] else "."
        return self.hallway[node] if self.hallway[node] else "."

    def __eq__(self, other: State) -> bool:
        return self.costs == other.costs and self.not_placed == other.not_placed

    def __lt__(self, other: State) -> bool:
        if self.costs < other.costs:
            return True

        if sum(len(c) * self.Cost[a] for a, c in self.not_placed.items()) < sum(
            len(c) * self.Cost[a] for a, c in other.not_placed.items()
        ):
            return True
        return False

    def __hash__(self):
        return hash(
            (
                tuple(self.rooms.values()),
                tuple(self.hallway.items()),
            )
        )

    def __repr__(self) -> str:
        return " ".join(
            (
                f"{self.costs}",
                f"{','.join(f'{k}={v}'for k, v in self.rooms.items())}",
                f"{",".join(f"{k}={v}" for k, v in self.hallway.items())}",
                f"{", ".join(f"{k}={v}" for k, v in self.not_placed.items())})",
            )
        )

    def __str__(self) -> str:
        rows = range(len(self.diagram))
        cols = range(len(self.diagram[0]))
        return "\n".join(
            (
                f"state.rooms = {{ {', '.join(f'\'{k}\':{f'\'{v}\'' if v else None}'for k, v in self.rooms.items())} }}",
                f"state.hallway = {{ {", ".join(f"\'{k}\':{f'\'{v}\'' if v else None}" for k, v in self.hallway.items())} }}",
                f"state.costs = {self.costs}",
                f"state.not_placed = {{ {", ".join(f"\'{k}\':{v}" for k, v in self.not_placed.items())} }}",
                f"\n".join("".join(self._get_str(r, c) for c in cols) for r in rows),
                "==" * 40,
            )
        )


class Puzzle:
    def __init__(self, s: str) -> None:
        self.diagram = [list(l) for l in s.splitlines()[1:]]
        self.state = State(self.diagram)

    def organize(self, verbose=False) -> int:
        heap = [self.state]
        seen = set()

        while heap:
            state = heappop(heap)

            if state.is_goal():
                return state.costs

            if state in seen:
                continue

            seen.add(state)

            for s1 in state.move(verbose):
                heappush(heap, s1)

        return -1

    def __repr__(self) -> str:
        rows = range(len(self.diagram))
        cols = range(len(self.diagram[0]))
        return "\n".join(
            (
                f"State = {self.state}: \n",
                f"\n".join("".join(self._get_str(r, c) for c in cols) for r in rows),
            )
        )

    def _get_str(self, r, c) -> str:
        if self.diagram[r][c] == "#":
            return "#"

        if (r, c) not in self.Points_Nodes:
            return "."

        node = self.PointsNodes[(r, c)]
        if len(node) == 2:
            return self.state.rooms[node] if self.state.rooms[node] else "."
        return self.state.hallway[node] if self.state.hallway[node] else "."


# test 1
# assert Puzzle(example).organize() == 12521
Puzzle(example).organize()

In [ ]:
puzzle = """
#############
#...........#
###D#D#C#C###
  #B#A#B#A#  
  #########  
"""

Puzzle(puzzle).organize(True)

<link href="style.css" rel="stylesheet"></link>


In [10]:
# print(f"Part II: { Toboggan(puzzle).count_tree_on_path_down_on_slopes()}")

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3093068400</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
